In [5]:
import os
import sqlite3
import datetime
import socket
import numpy as np
import healpy as hp
import pandas as pd
import matplotlib.path as mplPath
from rubin_sim.utils import (
    _hpid2_ra_dec,
    xyz_angular_radius,
    _build_tree,
    _xyz_from_ra_dec,
)
from rubin_sim.scheduler.utils import hp_kd_tree

In [51]:
class HpInLsstFov(object):
    """
    Return the healpixels within a pointing. A very simple LSST camera model with
    no chip/raft gaps.
    """

    def __init__(self, nside=None, fov_radius=1.75, scale=1e5):
        """
        Parameters
        ----------
        fov_radius : float (1.75)
            Radius of the filed of view in degrees
        scale : float (1e5)
            How many sig figs to round off to. Useful for ensuring identical results
            cross-ploatform where float precision can vary.
        """
        if nside is None:
            nside = set_default_nside()

        self.tree = hp_kd_tree(nside=nside, scale=scale)
        self.radius = np.round(xyz_angular_radius(fov_radius) * scale).astype(int)
        self.scale = scale

    def __call__(self, ra, dec, **kwargs):
        """
        Parameters
        ----------
        ra : float
            RA in radians
        dec : float
            Dec in radians

        Returns
        -------
        indx : numpy array
            The healpixels that are within the FoV
        """

        x, y, z = _xyz_from_ra_dec(ra, dec)
        x = np.round(x * self.scale).astype(int)
        y = np.round(y * self.scale).astype(int)
        z = np.round(z * self.scale).astype(int)
        
        if np.size(x) == 1:
            indices = self.tree.query_ball_point((np.max(x), np.max(y), np.max(z)), self.radius)
        else:
            indices = self.tree.query_ball_point(list(zip(x, y, z)), self.radius)
        return indices


In [52]:
ack = HpInLsstFov(nside=32)

In [53]:
print(ack([0.,0], [np.radians(-20), np.radians(-10)]))

[list([8128, 8255, 8256]) list([6976, 7104, 7231, 7232])]


In [54]:
print(ack([0.], [0.]))

[6207, 6208, 6080, 5952]


In [55]:
ack([0.], [0.])

[6207, 6208, 6080, 5952]

In [56]:
ack(0., 0.)

[6207, 6208, 6080, 5952]